In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
df

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [2]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')

In [3]:
df = df.rename(columns={"userId": "userId:token", "movieId": "movieId:token", "timestamp_int": "timestamp:float"})


In [4]:
test_part = 0.2
train_len = int(df.shape[0] * (1 - test_part))
test_len = df.shape[0] - train_len 
df_train, df_test = df.drop(['timestamp'], axis=1).head(train_len), df.drop(['timestamp'], axis=1).tail(test_len)

In [5]:
df_train.head()

,userId:token,movieId:token,rating
4182421,28507,1176,4.0
18950979,131160,1079,3.0
18950936,131160,47,5.0
18950930,131160,21,3.0
12341178,85252,45,3.0


In [6]:
N_USERS = df['userId:token'].max() + 1
N_ITEMS = df['movieId:token'].max() + 1

In [7]:
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [8]:
from math import log2

def apk(pred, target, k):
    if len(pred) >= k:
        pred = pred[:k]

    ans, cnt = 0, 0
    tot = len(pred) 
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            cnt += 1
            ans += cnt / (i + 1)
            s.add(pred[i])
    return ans / tot


def mapk(pred, target, k):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += apk(cur_pred, cur_target, k)
    return sum_metric / len(pred) 


def mr(pred, target):
    ans, cnt = 0, 0
    tot = len(pred)
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            ans += 1 / (i + 1)
            s.add(pred[i])
            break
    return ans

def mrr(pred, target):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += mr(cur_pred, cur_target)
    return sum_metric / len(pred) 


def ndcgunique(pred, target):
    ans, cnt = 0, 0
    tot = len(pred)
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            ans += 1 / (log2(i + 2))
            s.add(pred[i])
    return ans / tot

def ndcg(pred, target):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += ndcgunique(cur_pred, cur_target)
    return sum_metric / len(pred) 


def precisionunique(pred, target, k):
    if len(pred) >= k:
        pred = pred[:k]

    ans, cnt = 0, 0
    tot = len(pred)
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            ans += 1
            s.add(pred[i])
    return ans / tot


def precision(pred, target, k):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += precisionunique(cur_pred, cur_target, k)
    return sum_metric / len(pred) 

In [9]:
import queue
from tqdm.auto import trange


class GraphDataset(object):
    def __init__(self, df):
        self.df = df
        self.graph = [[] for _ in range(N_USERS + N_ITEMS)]
        for i in trange(self.df.shape[0]):
            usr, item = self.df['userId:token'].iloc[i], self.df['movieId:token'].iloc[i] + N_USERS
            self.graph[usr].append(item)
            self.graph[item].append(usr)

    

    def __getitem__(self, index):
        vertexes = set()
        edges = []
        q = queue.Queue()
        q.put((index, 0))
        while q.qsize() > 0:
            u, d = q.get()
            vertexes.add(u)
            if d >= 1:
                continue
            for v0 in self.graph[u]:
                if v0 not in vertexes:
                    edges.append((u, v0))
                    q.put((v0, d + 1))
        return list(vertexes), edges

    def __len__(self):
        return len(self.graph)


In [10]:
dataset = GraphDataset(df_train)

  0%|          | 0/16000210 [00:00<?, ?it/s]

In [11]:
sample_graph = dataset[1]
#sample_graph

In [12]:
import torch

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-geometric


Looking in links: https://data.pyg.org/whl/torch-1.11.0.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 828.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.0-cp37-cp37m-linux_x86_64.whl size=4187464 sha256=49aa770ab1ccc213beb37a8ea811e4834989873ea5cfe3050c753afb01cb4d01
  Stored in directory: /root/.cache/pip/wheels/2d/d1/15/8a2f0086896d156654a843fff4bdbeaf621cdd10310a0daad2
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.11.0.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.16-cp37-cp37m-linux_x86_64.whl size=1743105 sha256=944703ba77125ff1f4003d9e36e4bd7d4f567357c12047c3028f8af13180f7ef
  Stored in directory: /root/.cache/pip/wheels/92/8d/17/1517bf0318b8343b665b7aadf906dff4bc8967d82cacc17992
Successfully built torch-s

In [13]:
def renumber_graph(vertexes, edges):
    d = {v: i for i, v in enumerate(vertexes)}
    new_edges = [(d[u1], d[u2]) for u1, u2 in edges]
    new_vertexes = list(range(len(vertexes)))
    return new_vertexes, new_edges

In [14]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [15]:
import torch
import torch.nn as nn
import torch_geometric as tg

class SHAN(nn.Module):
    def __init__(self, n_users, n_items, hidden_dim):
        super().__init__()
        self.embedding_items = nn.Embedding(n_items, hidden_dim)
        self.embedding_users = nn.Embedding(n_users, hidden_dim)
        self.additional_info_encoding = nn.Linear(2, hidden_dim)

        self.attention_1 = nn.Linear(2 * hidden_dim, 1)
        self.attention_2 = nn.Linear(2 * hidden_dim, 1)
        self.last_layer = nn.Linear(hidden_dim, n_items)
        self.hidden_dim = hidden_dim

    def forward(self, history_input, metadata, user_id):
        len_long_history = int(history_input.shape[1] * (2/3))
        
        long_history, short_history = history_input[:, :len_long_history], history_input[:, len_long_history:]
        encoded_metadata = self.additional_info_encoding(metadata)
        encoded_long_history = self.embedding_items(long_history) + encoded_metadata[:, :len_long_history, :]
        encoded_short_history = self.embedding_items(short_history) + encoded_metadata[:, len_long_history:, :]
        encoded_user = self.embedding_users(user_id)

        encoded_long_user = torch.cat([encoded_user.unsqueeze(1)] * len_long_history, 1)
        weights_long = self.attention_1(torch.cat([encoded_long_history, encoded_long_user], 2)).softmax(1)
        weights_long = torch.cat([weights_long] * self.hidden_dim, 2)
        output_long = torch.sum(weights_long * encoded_long_history, 1).unsqueeze(1)

        encoded_short_history = torch.cat([encoded_short_history, output_long], 1)
        encoded_short_user = torch.cat([encoded_user.unsqueeze(1)] * encoded_short_history.shape[1], 1)
        #print(encoded_short_history.shape, encoded_short_user.shape)
        weights_short = self.attention_2(torch.cat([encoded_short_history, encoded_short_user], 2)).softmax(1)
        weights_short = torch.cat([weights_short] * self.hidden_dim, 2)
        output_short = torch.sum(weights_short * encoded_short_history, 1)
    
        return self.last_layer(output_short)


In [16]:
model_pizdim_embedding = torch.load('/kaggle/input/shan-model/model.pt').to(device)
for param in model_pizdim_embedding.parameters():
    param.requires_grad=False

In [17]:
import torch
import torch.nn as nn
import torch_geometric as tg


class NGCF(nn.Module):
    def __init__(self, n_vertexes, num_layers, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(n_vertexes, hidden_dim)
        for param in self.embedding.parameters():
            param.requires_grad=False

        for i in range(model_pizdim_embedding.embedding_users.weight.shape[0]):
            self.embedding.weight[i] = model_pizdim_embedding.embedding_users.weight[i]
            
        for i in range(model_pizdim_embedding.embedding_items.weight.shape[0]):
            self.embedding.weight[i + N_USERS] = model_pizdim_embedding.embedding_items.weight[i]
            
        for param in self.embedding.parameters():
            param.requires_grad=False
        self.graph_layer = nn.ModuleList([tg.nn.GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers)])
        self.relu = nn.ReLU()
        self.last = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, vertexes, edges):
        reordered_v, reordered_edges = renumber_graph(vertexes, edges)
        embeddings = self.embedding(torch.tensor(vertexes).unsqueeze(0).to(device))
        edges = torch.LongTensor(reordered_edges).to(device).T
        hidden = embeddings
        for i in range(len(self.graph_layer)):
            hidden = self.graph_layer[i](hidden, edges)
            hidden = self.relu(hidden)
        return self.last(hidden)

In [18]:
model = NGCF(N_ITEMS + N_USERS, 2, 64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.BCELoss()

In [19]:
vertex2interations = [[] for _ in range(N_USERS + N_ITEMS)]

for i in trange(df_train.shape[0]):
    usr, item = df_train['userId:token'].iloc[i], df_train['movieId:token'].iloc[i] + N_USERS
    rating = df['rating'].iloc[i]
    vertex2interations[usr].append((item, rating))
    vertex2interations[item].append((usr, rating))

  0%|          | 0/16000210 [00:00<?, ?it/s]

In [20]:
from tqdm.auto import trange, tqdm

epochs = 1

for _ in trange(epochs):
    for i in trange(1, len(dataset)):
        optimizer.zero_grad()
        batch = dataset[i]
        if len(batch[1]) == 0:
            continue
        output = model(*batch).squeeze(0)
        loss = 0
        d = {v: i for i, v in enumerate(batch[0])}
        for v, rating in vertex2interations[i]:
            inv_rating = 5 - rating
            cur_rating = torch.sum((output[d[i]] - output[d[v]]) * (output[d[i]] - output[d[v]]), -1)
            loss += (cur_rating - inv_rating) * (cur_rating - inv_rating)
        loss.backward()
        optimizer.step()

        

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/269756 [00:00<?, ?it/s]

In [21]:
torch.save(model, 'model.pt')

In [22]:
from math import log2

def apk(pred, target, k):
    if len(pred) >= k:
        pred = pred[:k]

    ans, cnt = 0, 0
    tot = len(pred) 
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            cnt += 1
            ans += cnt / (i + 1)
            s.add(pred[i])
    return ans / tot


def mapk(pred, target, k):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += apk(cur_pred, cur_target, k)
    return sum_metric / len(pred) 


def mr(pred, target):
    ans, cnt = 0, 0
    tot = len(pred)
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            ans += 1 / (i + 1)
            s.add(pred[i])
            break
    return ans

def mrr(pred, target):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += mr(cur_pred, cur_target)
    return sum_metric / len(pred) 


def ndcgunique(pred, target):
    ans, cnt = 0, 0
    tot = len(pred)
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            ans += 1 / (log2(i + 2))
            s.add(pred[i])
    return ans / tot

def ndcg(pred, target):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += ndcgunique(cur_pred, cur_target)
    return sum_metric / len(pred) 


def precisionunique(pred, target, k):
    if len(pred) >= k:
        pred = pred[:k]

    ans, cnt = 0, 0
    tot = len(pred)
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            ans += 1
            s.add(pred[i])
    return ans / tot


def precision(pred, target, k):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += precisionunique(cur_pred, cur_target, k)
    return sum_metric / len(pred) 

In [23]:


class FastKNN:
    def __init__(self, num_projections = 10):
        self.num_projections = num_projections
        self.distributions = []
        self.projection_matrixes = []
        self.embeddings = None

    def binary_search(self, index_projection, find_value):
        left_index, right_index = 0, len(self.distributions[index_projection]) - 1
        while left_index < right_index - 1:
            middle_index = (left_index + right_index) // 2
            middle_value = self.distributions[index_projection][middle_index][0]
            if middle_value < find_value:
                left_index = middle_index
            else:
                right_index = middle_index

        return left_index

    def find_k_nearest(self, index_projection, find_value, k):
        i0 = self.binary_search(index_projection, find_value)
        left_index = max(0, i0 - k // 2)
        right_index = min(len(self.distributions[index_projection]), left_index + k )
        if right_index - left_index != k:
            left_index = right_index - k
        return [self.distributions[index_projection][i][1] for i in range(left_index, right_index)]

    def fit(self, vectors):
        for i in trange(self.num_projections):
            projection = torch.randn((vectors.shape[1], 1))
            values = (vectors @ projection).cpu().numpy().reshape(-1)
            self.distributions.append(sorted([(values[j], j) for j in range(len(values))]))
            self.projection_matrixes.append(projection.cpu())

    def predict(self, vector, k):
        k0 = 1 + (k // self.num_projections)
        values = [torch.sum(vector * proj.reshape(-1)).item() for proj in self.projection_matrixes]
        neighb = []
        for i, value in enumerate(values):
            cur_n = self.find_k_nearest(i, value, k0)
            neighb += cur_n
        neighb = list(set(neighb))
        return neighb[:k]

In [24]:
import numpy as np

vectors = model.embedding.weight.cpu().detach()[N_USERS:, :]
knn = FastKNN()
knn.fit(vectors)

  0%|          | 0/10 [00:00<?, ?it/s]

In [25]:
g_test = [[] for i in range(df['userId:token'].max() + 1)]
for i in trange(df_test.shape[0]):
    usr, item = df_test['userId:token'].iloc[i], df_test['movieId:token'].iloc[i]
    g_test[usr].append(item)


  0%|          | 0/4000053 [00:00<?, ?it/s]

In [26]:
del vertex2interations

In [27]:
g_train = [[] for i in range(df_train['userId:token'].max() + 1)]
for i in trange(df_train.shape[0]):
    cur_user = df['userId:token'].iloc[i]
    cur_item = df['movieId:token'].iloc[i]
    cur_rating = df['rating'].iloc[i]
    g_train[cur_user].append((cur_rating, cur_item))

for i in range(len(g_train)):
    g_train[i] = sorted(g_train[i])
    g_train[i] = g_train[i][-5:]


  0%|          | 0/16000210 [00:00<?, ?it/s]

In [28]:
predictions = []
target = []
k = 40
for i in trange(df_train['userId:token'].max() + 1):
    if len(g_test[i]) == 0 or len(g_train[i]) == 0:
        continue
    out = []
    for l in g_train[i]:
        cur_item = l[1]
        neighb = knn.predict(vectors[cur_item], k)[:(k // 5)]
        for e in neighb:
            out.append(e)
    
    predictions.append(out)
    target.append(g_test[i])


  0%|          | 0/138494 [00:00<?, ?it/s]

In [29]:
df_test['userId:token'].value_counts()

15617     4354
130459    3908
31122     3742
92269     3456
105580    3437
          ... 
92533        1
16972        1
84182        1
57818        1
69883        1
Name: userId:token, Length: 31670, dtype: int64

In [30]:
df_test['userId:token'].min()

11

In [31]:
len(g_train)

138494

In [32]:
for i in range(1, 20):
    print(f"precision@{i} : {precision(predictions, target, i)}")
print(f"ndcg: {ndcg(predictions, target)}")
print(f"mrr: {mrr(predictions, target)}")
for i in range(1, 20):
    print(f"mapk@{i} : {mapk(predictions, target, i)}")


precision@1 : 0.010455431508063088
precision@2 : 0.012316143895091263
precision@3 : 0.013172662295469314
precision@4 : 0.013246500088605352
precision@5 : 0.01261740209108636
precision@6 : 0.012847776005670728
precision@7 : 0.013164223690539519
precision@8 : 0.012781321991848308
precision@9 : 0.012764093173450015
precision@10 : 0.012750310118731203
precision@11 : 0.012739033073961403
precision@12 : 0.012626262626262636
precision@13 : 0.012462683515315235
precision@14 : 0.012335130756183345
precision@15 : 0.012319097406816817
precision@16 : 0.012050327839801525
precision@17 : 0.012037297641012985
precision@18 : 0.011868194616732768
precision@19 : 0.011922083251723274
ndcg: 0.0030767777247616497
mrr: 0.03772437776541124
mapk@1 : 0.010455431508063088
mapk@2 : 0.009214956583377636
mapk@3 : 0.007915411424183356
mapk@4 : 0.006966595782385257
mapk@5 : 0.0061190855927698035
mapk@6 : 0.005645637663181539
mapk@7 : 0.005280336483344005
mapk@8 : 0.004902723981671345
mapk@9 : 0.004575593257367142
ma